In [26]:
import requests
import json
import time
from datetime import datetime
import os
import gzip

def get_futures_orderbook(symbol="BTCUSDT", limit=20):
    """
    Obtém o livro de ordens do mercado futuro da Binance para o par especificado.
    """
    base_url = "https://fapi.binance.com/fapi/v1/depth"
    params = {"symbol": symbol, "limit": limit}
    
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Erro ao obter o livro de ordens: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Erro na requisição: {e}")
        return None


def ensure_valid_gzip(filename):
    """
    Garante que o arquivo gzip seja válido. Se não existir, cria um arquivo inicial vazio compactado.
    """
    if not os.path.exists(filename):
        # Cria um arquivo gzip vazio contendo uma lista
        with gzip.open(filename, "wt") as f:
            json.dump([], f)


def load_existing_data(filename):
    """
    Carrega os dados existentes de um arquivo compactado gzip.
    """
    try:
        with gzip.open(filename, "rt") as f:
            return json.load(f)
    except (OSError, json.JSONDecodeError):
        print(f"Erro ao carregar o arquivo {filename}. Inicializando como lista vazia.")
        return []


def save_orderbook_to_json(data, filename="futures_orderbook_history.json.gz"):
    """
    Salva o livro de ordens em um arquivo JSON compactado.
    """
    try:
        # Garante que o arquivo gzip exista
        ensure_valid_gzip(filename)

        # Carrega os dados existentes
        existing_data = load_existing_data(filename)

        # Adiciona o novo snapshot
        existing_data.append(data)

        # Salva os dados atualizados no arquivo compactado
        with gzip.open(filename, "wt") as f:
            json.dump(existing_data, f)
        
        # print(f"Snapshot salvo em {filename}")
    except Exception as e:
        print(f"Erro ao salvar o livro de ordens: {e}")


def collect_futures_orderbook_continuously(symbol="BTCUSDT", limit=20, interval=5, duration=60, filename="futures_orderbook_history.json.gz"):
    """
    Coleta snapshots do livro de ordens continuamente em intervalos regulares.
    """
    end_time = time.time() + duration
    while time.time() < end_time:
        # Obter o livro de ordens
        snapshot = get_futures_orderbook(symbol, limit)
        if snapshot:
            # Adicionar timestamp ao snapshot
            snapshot["timestamp"] = datetime.utcnow().isoformat()
            # Salvar o snapshot no arquivo compactado
            save_orderbook_to_json(snapshot, filename)
        time.sleep(interval)



In [ ]:

# Coletar dados por 1 minuto (60 segundos), com snapshots a cada 5 segundos
collect_futures_orderbook_continuously(symbol="VIDTUSDT", limit=10, interval=2, duration=86400, filename="futures_orderbook_history.json.gz")


/tmp/ipykernel_4969/3464407049.py:83: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  snapshot["timestamp"] = datetime.utcnow().isoformat()


In [ ]:
comecei as 17:17 dia  23 dezembro

In [18]:
import json
from collections import defaultdict
from decimal import Decimal, ROUND_DOWN

def group_orderbook_by_tick(orderbook_snapshots, tick_size):
    """
    Processa o JSON do orderbook e agrupa os preços por um tick size especificado.

    :param orderbook_snapshots: Lista de snapshots do orderbook.
    :param tick_size: Tamanho do tick (ex.: 0.01, 0.001, 0.0001).
    :return: Lista de snapshots agrupados pelo tick size.
    """
    tick_size = Decimal(str(tick_size))  # Garantir precisão com decimal
    grouped_snapshots = []

    for snapshot in orderbook_snapshots:
        grouped_snapshot = {
            "timestamp": snapshot["timestamp"],
            "bids": _group_prices(snapshot["bids"], tick_size),
            "asks": _group_prices(snapshot["asks"], tick_size)
        }
        grouped_snapshots.append(grouped_snapshot)
    
    return grouped_snapshots


def _group_prices(prices, tick_size):
    """
    Agrupa preços de bids ou asks por um tick size especificado.

    :param prices: Lista de preços e quantidades (ex.: [["0.0320400", "10245"], ...]).
    :param tick_size: Tamanho do tick (ex.: 0.01, 0.001, 0.0001).
    :return: Lista agrupada de preços e quantidades.
    """
    grouped = defaultdict(Decimal)

    for price, quantity in prices:
        # Arredonda o preço para o tick size
        rounded_price = (Decimal(price) // tick_size) * tick_size
        # Soma a quantidade no preço arredondado
        grouped[rounded_price] += Decimal(quantity)
    
    # Retorna o resultado como uma lista ordenada
    return [[str(price), str(quantity)] for price, quantity in sorted(grouped.items())]



In [25]:
filename = "futures_orderbook_history.json.gz"

# Garante que o arquivo gzip exista
ensure_valid_gzip(filename)

# Carrega os dados existentes
existing_data = load_existing_data(filename)


# Agrupar bids e asks com tick size de 0.01
tick_size = 0.001
grouped = group_orderbook_by_tick(existing_data, tick_size)


# Salvar ou imprimir o resultado
print(json.dumps(grouped, indent=4))


[
    {
        "timestamp": "2024-12-23T19:56:58.464311",
        "bids": [
            [
                "0.030",
                "874000"
            ],
            [
                "0.031",
                "13078333"
            ],
            [
                "0.032",
                "319998"
            ]
        ],
        "asks": [
            [
                "0.032",
                "7611041"
            ],
            [
                "0.033",
                "2034301"
            ]
        ]
    },
    {
        "timestamp": "2024-12-23T19:57:08.335678",
        "bids": [
            [
                "0.030",
                "720609"
            ],
            [
                "0.031",
                "12602135"
            ],
            [
                "0.032",
                "608868"
            ]
        ],
        "asks": [
            [
                "0.032",
                "7329090"
            ],
            [
                "0.033",
                "20